# Springboard - Data Science Career Track - Capstone 1

In [242]:
# Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.metrics import mean_absolute_error
from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV

In [243]:
# Read in data from CSV
df = pd.read_csv('allCongressDataPublish.csv')

# Inspect the data
df.head()

,stateDist,abroadPrcnt,recentArrivalPrcnt,totalPopBirthPlace,prcntForeignBorn,prcntExAliens,totalHouseholds,under10k,over10k,over15k,...,numPassH,numEnact,passPrcnt,congNum,partyControl,demSeats,repSeats,otherSeats,ses,sesNorm
0,AK.0,NaN,54.541518,588358.0,1.327253,NaN,106181.0,14.029817,85.970183,78.523465,...,3.0,2.0,6.666667,93.0,D,243.0,192.0,0.0,0.147659,35.339666
1,AL.1,NaN,15.405078,543377.0,0.457325,NaN,148712.0,20.154392,79.845608,72.468261,...,0.0,0.0,0.000000,93.0,D,243.0,192.0,0.0,-1.075380,11.690437
2,AL.2,NaN,24.197693,530286.0,0.495582,NaN,165462.0,23.441636,76.558364,67.187632,...,2.0,1.0,3.030303,93.0,D,243.0,192.0,0.0,-1.097884,11.255282
3,AL.3,NaN,20.234837,555795.0,0.317203,NaN,162704.0,25.357090,74.642910,66.138509,...,1.0,0.0,0.000000,93.0,D,243.0,192.0,0.0,-1.264190,8.039513
4,AL.4,NaN,15.244589,493752.0,0.148253,NaN,159228.0,21.855453,78.144547,69.694400,...,1.0,1.0,5.882353,93.0,D,243.0,192.0,0.0,-1.435099,4.734744


In [244]:
# Delete unneeded columns
del df['abroadPrcnt']
del df['prcntExAliens']
del df['totalHouseholds']
del df['meanIncome']
del df['medianIncome']
del df['totalEmploymentPop']
del df['prcntNotEmploy']
del df['totalPopRaceFile']
del df['prcntBlackNotHisp']
del df['prcntMulti']
del df['prcntWhite']
del df['prcntNotHisp']
del df['prcntOld']
del df['medianAge']
del df['sponID']
del df['icpsr']
del df['state']
del df['district']
del df['lastName']
del df['firstName']
del df['middleName']
del df['age']
del df['gender']
del df['numberTerms']
del df['daysServed']
del df['fracServed']
del df['cd']
del df['statenm']
del df['dwnom1']
del df['dwnom2']
del df['comPower']
del df['chair']
del df['rank']
del df['comName']
del df['numCom']
del df['rankChair']
del df['black']
del df['hispanic']
del df['numSpon']
del df['numCosp']
del df['numPassH']
del df['numEnact']
del df['passPrcnt']
del df['partyControl']
del df['demSeats']
del df['repSeats']
del df['otherSeats']
del df['ses']
del df['sesNorm']

# Describe the data
df.describe()

,recentArrivalPrcnt,totalPopBirthPlace,prcntForeignBorn,under10k,over10k,over15k,over25k,over35k,over50k,over75k,...,over200k,prcntUnemp,prcntBA,prcntHS,prcntAsian,prcntBlack,prcntHisp,prcntWhiteAll,gini,congNum
count,9312.000000,9.312000e+03,9312.000000,9312.000000,9312.000000,9312.000000,9312.000000,9312.000000,9312.000000,9312.000000,...,9312.000000,9312.000000,9312.000000,9312.000000,9312.000000,9312.000000,9312.000000,9312.000000,9312.000000,9410.000000
mean,16.199663,8.279402e+05,8.245649,9.390199,90.607782,83.863417,74.306194,61.674934,45.942324,27.715459,...,2.168106,4.760931,19.946112,71.517403,2.952052,11.386237,9.980803,72.109506,0.370931,103.015515
std,9.428621,1.011553e+06,9.837913,5.637967,5.640051,6.877721,8.529183,10.369699,11.900707,10.949647,...,2.508589,2.915304,10.374466,15.559808,5.531153,14.646916,14.627979,23.576277,0.157564,6.057714
min,0.700000,3.766190e+05,0.074021,1.344659,66.998284,56.001452,43.200000,29.205036,16.490626,6.400000,...,0.041587,1.192474,1.906013,24.586055,0.004082,0.012149,0.166042,3.158279,0.032949,93.000000
25%,7.300000,5.309882e+05,1.809899,5.132383,87.569965,80.000000,68.600000,54.400000,37.385944,19.400000,...,0.715248,2.736828,11.886034,59.648148,0.343950,2.300000,1.434007,61.607353,0.372793,98.000000
50%,16.000000,6.394565e+05,4.484447,8.000000,92.017785,85.125675,75.100000,61.599035,45.068233,25.998169,...,1.301726,3.800000,18.100000,75.200000,1.151466,5.641133,3.702759,79.285090,0.439405,103.000000
75%,22.562300,7.149890e+05,10.500000,12.461961,94.900000,88.700000,80.453940,68.634682,53.552165,34.254053,...,2.600000,6.000000,25.600000,84.800000,3.000000,14.000000,12.034055,90.200465,0.475321,108.000000
max,54.858657,1.401422e+07,60.800000,33.001716,98.655341,97.100000,94.012948,90.500000,82.400000,68.700000,...,22.200000,27.500000,68.800000,96.600000,68.100000,92.111006,87.700000,99.399399,0.607980,113.000000


In [245]:
# Remove rows with missing values
df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)

# Remove 'Independent' rows contained in 'party' from the data 
df = df[df['party'] != 'Independent']

# Replace 'Republican-Conservative' rpws with 'Republican' in the data
df = df.replace(to_replace='Republican-Conservative', value='Republican')

# Convert non-numerical data using one-hot encoding
df = pd.get_dummies(df, columns=['party'])

# Delete the 'party_Republican' column
del df['party_Republican']

df.head()

,stateDist,recentArrivalPrcnt,totalPopBirthPlace,prcntForeignBorn,under10k,over10k,over15k,over25k,over35k,over50k,...,prcntUnemp,prcntBA,prcntHS,prcntAsian,prcntBlack,prcntHisp,prcntWhiteAll,gini,congNum,party_Democrat
0,AK.0,54.541518,588358.0,1.327253,14.029817,85.970183,78.523465,72.888747,63.228826,49.549354,...,4.485170,14.140283,66.658073,3.148423,1.514894,1.530580,40.675065,0.128563,93.0,0
1,AL.1,15.405078,543377.0,0.457325,20.154392,79.845608,72.468261,64.847490,51.344881,31.147453,...,2.692152,6.830163,40.297523,0.276788,29.581856,1.632954,60.562372,0.064529,93.0,0
2,AL.2,24.197693,530286.0,0.495582,23.441636,76.558364,67.187632,58.843118,44.934789,27.204434,...,1.877222,8.356176,43.119566,0.117107,27.597372,0.820256,64.945143,0.060138,93.0,0
3,AL.3,20.234837,555795.0,0.317203,25.357090,74.642910,66.138509,58.930942,44.303152,25.725858,...,2.038250,6.874170,36.865034,0.072509,27.756277,1.018866,60.953049,0.052972,93.0,1
4,AL.4,15.244589,493752.0,0.148253,21.855453,78.144547,69.694400,61.685131,46.047178,25.332228,...,2.535659,4.100965,32.548181,0.028962,8.503662,1.062788,91.135226,0.047624,93.0,1


In [246]:
# Create X and y arrays from the dataset
y = df['party_Democrat'].as_matrix()

del df['party_Democrat']

X = df.as_matrix()

In [247]:
# Split data into test/train set (70/30 split) and shuffle
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)